In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/MyDrive/ML datasets'
%ls

import warnings
warnings.filterwarnings('ignore')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/ML datasets
 abalone.data    DT-B-2-CC.png             mbti_1.csv          output_MLP.csv
 data_1.csv      DT-B-2-XX.png             output_cnn.csv      output_svm.csv
 data_2.csv      DT_C_1.pkl                output.csv         'ROC Curve.png'
 diabetes2.csv   fashion-mnist_test.csv    output_forest.csv   test_data.csv
 DT_A_1.png      fashion-mnist_train.csv   output_knn.csv      train_data.csv
 DT_B_1.png      imagename.png             output_log.csv


In [ ]:
#install packages
!pip install imbalanced-learn

#importing libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier



In [ ]:
#loading the training dataset
data=pd.read_csv('train_data.csv')

#loading the testing dataset
data_test=pd.read_csv('test_data.csv')

#storing IDs from testing dataset
test_data_id= []
for id,seq in zip(data_test['ID'], data_test['Sequence']):
    test_data_id.append(id)

In [ ]:
#Pre-Processing

#Feature Extraction
#Function to Add Features: Integer Encoding, Binary Profiling, Molecular_weight
char_dict= {'A': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'K': 9, 'L': 10, 'M': 11, 'N': 12, 'P': 13, 'Q': 14, 'R': 15, 'S': 16, 'T': 17, 'V': 18, 'W': 19, 'Y': 20}
molecular_weight_dict={"X":0,"A":89.1,"R":174.2,"N":132.1,"D":133.1,"C":121.2,"E":147.1,"Q":146.2,"G":75.1,"H":155.2,"I":131.2,"L":131.2,"K":146.2,"M":149.2,"F":165.2,"P":115.1,"S":105.1,"T":119.1,"W":204.2,"Y":181.2,"V":117.1}
distinct_genes='ACDEFGHIKLMNPQRSTVWXY'
def feature_extraction(data):
  encode_list = []
  for row in data['Sequence'].values:
    row_encode = []
    temp_array=[]
    weight=0
    for d in distinct_genes:
      temp_array.append(row.count(d))
    for code in row:
      row_encode.append(char_dict.get(code, 0))
      gene_string=[0 for k in range(21)]
      # print(i)
      gene_string[distinct_genes.index(code)]=1
      temp_array.extend(gene_string)
      # if not molecular_weight_dict.get(code):print(code)
      weight+=molecular_weight_dict.get(code)
    row_encode.extend(temp_array)
    row_encode.append(weight)
    encode_list.append(np.array(row_encode))
    # print(row_encode)
  return encode_list

In [ ]:
#Features and Labels splitting
X_data = feature_extraction(data)
Y_data = data['label'].values
X_data_test = feature_extraction(data_test)
#print(X_data[0])

In [ ]:
#Sampling:
under = RandomUnderSampler(sampling_strategy=0.7)   #Under sampling data to Fix Data imbalance 
X_data, Y_data = under.fit_resample(X_data, Y_data)
#print(Counter(Y_data))

In [ ]:
#Splitting data into train and validation dataset to check accuracy
X_train, X_val, Y_train, Y_val = train_test_split(X_data, Y_data, test_size=0.2, random_state=123,stratify= Y_data)

In [ ]:
#Best model: RandomForestRegressor

print("Best model: RandomForestRegressor: \n")
#min_samples_split=10 and n_estimators=100 to avoid overfitting and underfitting
rf = RandomForestRegressor(min_samples_split=10, n_estimators=100, n_jobs=-1) 

#GridSearchCV to apply  K-fold cross Validation and Hypeparameter tuning
grid = GridSearchCV(estimator = rf, param_grid = {}, cv = 10, verbose=2, n_jobs = -1)
grid.fit(X_data, Y_data)

#Best estimator from GridSearchCV
print(grid.best_estimator_)
forest= grid.best_estimator_

#Predicting values
predictions = forest.predict(X_data_test)

#Converting output into csv file
output_data= []
for i in range(len(predictions)):
    output_data.append([test_data_id[i],predictions[i]])
output_df=pd.DataFrame(output_data,columns=['ID','Label'])
output_df.to_csv('output.csv', sep=',', index=False)
print("\n Output File: \n")
print(output_df)
print("\n\n\n")

Best model: RandomForestRegressor: 

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.9min finished


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=10, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

 Output File: 

         ID     Label
0     10001  0.258232
1     10002  0.256519
2     10003  0.199651
3     10004  0.420792
4     10005  0.300917
...     ...       ...
9577  19578  0.509828
9578  19579  0.730983
9579  19580  0.336750
9580  19581  0.429539
9581  19582  0.393197

[9582 rows x 2 columns]






In [ ]:
#Model: Logistic Regression

print("Model: Logistic Regression: \n")

log = LogisticRegression(n_jobs=-1) 

#GridSearchCV to apply  K-fold cross Validation and Hypeparameter tuning
grid = GridSearchCV(estimator = log, param_grid = {}, cv = 10, verbose=2, n_jobs = -1)
grid.fit(X_train, Y_train)

#Best estimator from GridSearchCV
print(grid.best_estimator_)
logistic= grid.best_estimator_

#Chceking accuracy on val data
Y_pred_val= logistic.predict(X_val)
print("\n Classification Report for Validation dataset: \n")
print(classification_report(Y_val, Y_pred_val))


#Predicting values
predictions = logistic.predict(X_data_test)

#Converting output into csv file
output_data= []
for i in range(len(predictions)):
    output_data.append([test_data_id[i],predictions[i]])
output_df=pd.DataFrame(output_data,columns=['ID','Label'])
output_df.to_csv('output_log.csv', sep=',', index=False)
print("\n Output File: \n")
print(output_df)
print("\n\n\n")

Model: Logistic Regression: 

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    2.3s finished


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=-1, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

 Classification Report for Validation dataset: 

              precision    recall  f1-score   support

           0       0.65      0.83      0.73       515
           1       0.59      0.37      0.45       360

    accuracy                           0.64       875
   macro avg       0.62      0.60      0.59       875
weighted avg       0.63      0.64      0.61       875


 Output File: 

         ID  Label
0     10001      0
1     10002      1
2     10003      0
3     10004      0
4     10005      0
...     ...    ...
9577  19578      0
9578  19579      1
9579  19580      0
9580  19581      0
9581  19582      0

[9582 rows x 2 columns]






In [ ]:
#Model: SVM

print("Model: SVM: \n")
svm = make_pipeline(StandardScaler(), SVC(gamma='auto'))

#training the model
svm.fit(X_train, Y_train)

#Chceking accuracy on val data
Y_pred_val= svm.predict(X_val)
print("\n Classification Report for Validation dataset: \n")
print(classification_report(Y_val, Y_pred_val))

#Predicting values
predictions = svm.predict(X_data_test)

#Converting output into csv file
output_data= []
for i in range(len(predictions)):
    output_data.append([test_data_id[i],int(predictions[i])])
output_df=pd.DataFrame(output_data,columns=['ID','Label'])
output_df.to_csv('output_svm.csv', sep=',', index=False)
print("\n Output File: \n")
print(output_df)
print("\n\n\n")

Model: SVM: 


 Classification Report for Validation dataset: 

              precision    recall  f1-score   support

           0       0.68      0.92      0.78       515
           1       0.76      0.38      0.51       360

    accuracy                           0.70       875
   macro avg       0.72      0.65      0.65       875
weighted avg       0.71      0.70      0.67       875


 Output File: 

         ID  Label
0     10001      0
1     10002      0
2     10003      0
3     10004      0
4     10005      0
...     ...    ...
9577  19578      0
9578  19579      1
9579  19580      0
9580  19581      0
9581  19582      1

[9582 rows x 2 columns]






In [ ]:
#Model: Kth Nearest Neighbour

print("Model: KNN \n")
knn = KNeighborsClassifier(n_neighbors=20)

#training the model
knn.fit(X_train, Y_train)

#Chceking accuracy on val data
Y_pred_val= knn.predict(X_val)
print("\n Classification Report for Validation dataset: \n")
print(classification_report(Y_val, Y_pred_val))

#Predicting values
predictions = knn.predict(X_data_test)

#Converting output into csv file
output_data= []
for i in range(len(predictions)):
    output_data.append([test_data_id[i],int(predictions[i])])
output_df=pd.DataFrame(output_data,columns=['ID','Label'])
output_df.to_csv('output_svm.csv', sep=',', index=False)
print("\n Output File: \n")
print(output_df)
print("\n\n\n")

Model: KNN 


 Classification Report for Validation dataset: 

              precision    recall  f1-score   support

           0       0.60      0.89      0.72       515
           1       0.48      0.15      0.23       360

    accuracy                           0.58       875
   macro avg       0.54      0.52      0.47       875
weighted avg       0.55      0.58      0.52       875


 Output File: 

         ID  Label
0     10001      0
1     10002      0
2     10003      1
3     10004      0
4     10005      0
...     ...    ...
9577  19578      0
9578  19579      1
9579  19580      0
9580  19581      0
9581  19582      0

[9582 rows x 2 columns]






In [ ]:
#Best model:  RandomForestClassifier

print("Model:  RandomForestClassifier \n")
#min_samples_split=10 and n_estimators=100 to avoid overfitting and underfitting
rc = RandomForestClassifier(min_samples_split=10, n_estimators=100, n_jobs=-1) 

#GridSearchCV to apply  K-fold cross Validation and Hypeparameter tuning
grid = GridSearchCV(estimator = rc, param_grid = {}, cv = 10, verbose=2, n_jobs = -1)
grid.fit(X_train, Y_train)

#Best estimator from GridSearchCV
print(grid.best_estimator_)
forestc= grid.best_estimator_

#Chceking accuracy on val data
Y_pred_val= forestc.predict(X_val)
print("\n Classification Report for Validation dataset: \n")
print(classification_report(Y_val, Y_pred_val))


#Predicting values
predictions = forestc.predict(X_data_test)

#Converting output into csv file
output_data= []
for i in range(len(predictions)):
    output_data.append([test_data_id[i],predictions[i]])
output_df=pd.DataFrame(output_data,columns=['ID','Label'])
output_df.to_csv('output.csv', sep=',', index=False)
print("\n Output File: \n")
print(output_df)
print("\n\n\n")

Model:  RandomForestClassifier 

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    9.6s finished


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

 Classification Report for Validation dataset: 

              precision    recall  f1-score   support

           0       0.67      0.93      0.78       515
           1       0.79      0.36      0.49       360

    accuracy                           0.69       875
   macro avg       0.73      0.64      0.64       875
weighted avg       0.72      0.69      0.66       875


 Output File: 

         ID  Label
0     10001      0
1     10002      0
2  